In [1]:
# Install Pinecone client
!pip install pinecone-client

# Install SentenceTransformer for embedding generation
!pip install sentence-transformers

# Install Google Generative AI (Gemini-pro) client
!pip install google-generativeai

# Install pdfplumber for extracting text from PDFs
!pip install pdfplumber



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 95.4 MB/s eta 0:00:00


In [2]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import pdfplumber

In [3]:
# Configure Google Gemini-pro API
genai.configure(api_key="your_google_api_key")    # Replace with your actual api key

# Initialize Pinecone
pc = Pinecone(api_key="your_pinecone_api_key")    # Replace with your actual api key

In [4]:
index_name = "colab"             # Replace with your actual pinecone index name
index = pc.Index(index_name)

# Load embedding model
embedder = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_file):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

In [6]:
# Function to store document in Pinecone
def store_document_in_pinecone(doc_text):
    sentences = doc_text.split('. ')
    embeddings = embedder.encode(sentences)
    for i, emb in enumerate(embeddings):
        index.upsert([(f"sentence-{i}", emb, {'text': sentences[i]})])

In [7]:
# Function to retrieve relevant chunks from Pinecone
def retrieve_relevant_chunks(query, top_k=5):
    query_embedding = embedder.encode([query])
    results = index.query(vector=query_embedding.tolist(), top_k=top_k, include_metadata=True)
    relevant_chunks = [match['metadata']['text'] for match in results['matches']]
    return " ".join(relevant_chunks)

In [25]:
# Function to get an answer using Gemini-pro
def question_text(retrieved_text, question):
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content([f"Answer the following question:\n\nText: {retrieved_text}\n\nQuestion: {question}"])

    # Clean the generated response by removing asterisks
    cleaned_response = response.text.replace('*', '').replace('**', '')  # Remove asterisks

    return cleaned_response

In [26]:
# Load a PDF and store it in Pinecone
pdf_path = "/content/BlackHoles.pdf"    #Replace with you pdf file path
doc_text = extract_text_from_pdf(pdf_path)
store_document_in_pinecone(doc_text)


In [28]:
# Ask a question
question = "Who is Schwarchild?"
retrieved_text = retrieve_relevant_chunks(question)
answer = question_text(retrieved_text, question)
print("Retrieved Text:\n", retrieved_text)
print("\n\nGenerated Answer:\n", answer)


Retrieved Text:
 Hamilton]Karl Schwarzschild’s Work
In 1916 Schwarzschild read Einstein’s
paper on general relativity He was
interested in the physics of stars,
and had a lot of spare time between
battles on the Russian front, so he solved
Einstein’s field equation for the region
outside a massive spherical object.
His solution had many interesting features,
including
q prediction of space warping in strong
gravity, and invention of embedding
diagrams to visualize it.
q verification gravitational time dilation,
just as Einstein had pictured it.
q prediction of black holes, though this
[slide courtesy of D Watson]
was not recognized at the time.Schwarzschild’s solution
Describes the spacetime curvature near a massive, spherically
symmetric body Begelman & M Watson - Image from Thorne’s “Black Holes and time Warps”]


Generated Answer:
 The text states that Karl Schwarzschild is the person being discussed. 

